In [1]:
import pandas as pd
import os
import warnings

# Suprimir advertencias de conversión de fechas
warnings.simplefilter(action='ignore', category=UserWarning)

# Definir las rutas de los archivos
input_dir = r"C:\Users\Joseph´S - PC\OneDrive\Escritorio\Proyecto Henry 2"
output_dir = r"C:\Users\Joseph´S - PC\OneDrive\Escritorio\Proyecto Henry 2\Procesados"

# Obtener lista de archivos .xlsx en el directorio
xlsx_files = [f for f in os.listdir(input_dir) if f.endswith('.xlsx')]

# Asegurarse de que el directorio de salida existe
os.makedirs(output_dir, exist_ok=True)

print()



In [2]:
for file in xlsx_files:
    # Leer el archivo .xlsx en un diccionario de DataFrames (cada hoja se convierte en un DataFrame)
    file_path = os.path.join(input_dir, file)
    df_sheets = pd.read_excel(file_path, sheet_name=None)  # Carga todas las hojas
    
    # Crear un diccionario para almacenar las hojas procesadas
    processed_sheets = {}
    
    # Paso 2: Convertir columnas numéricas y de fecha para cada hoja
    for sheet_name, df in df_sheets.items():
        # Procesar las columnas del DataFrame
        for column in df.columns:
            try:
                # Intentar convertir a numérico
                df[column] = pd.to_numeric(df[column])
            except ValueError:
                pass  # Si no es posible convertir a numérico, no hacer nada

            try:
                # Intentar convertir a fecha
                df[column] = pd.to_datetime(df[column])
            except ValueError:
                pass  # Si no es posible convertir a fecha, no hacer nada
        
        # Almacenar la hoja procesada en el diccionario
        processed_sheets[sheet_name] = df
    
    # Paso 3: Guardar todas las hojas procesadas en un nuevo archivo .xlsx
    processed_file_path = os.path.join(output_dir, f"procesado_{file}")
    with pd.ExcelWriter(processed_file_path, engine='xlsxwriter') as writer:
        for sheet_name, processed_df in processed_sheets.items():
            processed_df.to_excel(writer, sheet_name=sheet_name, index=False)


In [3]:
import pandas as pd
import os
import warnings

# Suprimir advertencias de conversión de fechas
warnings.simplefilter(action='ignore', category=UserWarning)

# Definir las rutas de los archivos
input_dir = r"C:\Users\Joseph´S - PC\OneDrive\Escritorio\Proyecto Henry 2"

# Obtener lista de archivos .xlsx en el directorio y omitir los que contienen '(1)' en su nombre
xlsx_files = [f for f in os.listdir(input_dir) if f.endswith('.xlsx') and "(1)" not in f]

# Procesar cada archivo
for file in xlsx_files:
    file_path = os.path.join(input_dir, file)
    try:
        # Leer todas las hojas del archivo
        excel_data = pd.ExcelFile(file_path)
        print(f"Archivo: {file}")
        
    except Exception as e:
        print(f"Error al procesar el archivo {file}: {e}")




Archivo: Internet.xlsx
Archivo: mapa_conectividad.xlsx
Archivo: Portabilidad.xlsx
Archivo: servicios_postales.xlsx
Archivo: telefonia_fija.xlsx
Archivo: Telefonia_movil.xlsx
Archivo: Television.xlsx
